In [ ]:
import sys,os

In [ ]:
!{sys.executable} -m pip install -U sagemaker 
!{sys.executable} -m pip install -U sagemaker-experiments

In [ ]:
import time
from time import strftime

import sagemaker
from sagemaker.tensorflow import TensorFlow

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

from tensorflow.keras.datasets.fashion_mnist import load_data
import numpy as np

In [ ]:
# 全削除

for exp in Experiment.list():
    tmp_exp = Experiment.load(exp.experiment_name)
    tmp_exp.delete_all(action="--force")

In [ ]:
role = sagemaker.get_execution_role()
sm_sess = sagemaker.session.Session()

In [ ]:
project_name = 'fassion-mnist'
create_date = strftime("%Y%m%d-%H%M%S")
experiment = Experiment.create(experiment_name = f"{project_name}-{create_date}",
                               description = f"{project_name} experiment",
                               tags = [{'Key': f'{project_name}-experiments', 'Value': f'{project_name}_{create_date}'}])

In [ ]:
demo_trial = Trial.create(trial_name = f"{project_name}-{create_date}",
                          experiment_name = experiment.experiment_name,
                          tags = [{'Key': f'{project_name}-trials', 'Value': '1'}])

In [ ]:
(train_X, train_y), (test_X, test_y) = load_data()
# from -1 to 1 normalization and channel last
train_X = (train_X/127.5-1).astype(np.float32).reshape((train_X.shape[0],train_X.shape[1],train_X.shape[2],1))
test_X = (test_X/127.5-1).astype(np.float32).reshape((test_X.shape[0],test_X.shape[1],test_X.shape[2],1))
# one hot encoding
train_y = np.eye(10)[train_y]
test_y = np.eye(10)[test_y]

In [ ]:
# export npy
data_dir = './data/'
!mkdir -p {data_dir}
TRAIN_X_PATH = os.path.join(data_dir,'train_X.npy')
TEST_X_PATH = os.path.join(data_dir,'test_X.npy')
TRAIN_Y_PATH = os.path.join(data_dir,'train_y.npy')
TEST_Y_PATH = os.path.join(data_dir,'test_y.npy')
np.save(TRAIN_X_PATH,train_X)
np.save(TEST_X_PATH,test_X)
np.save(TRAIN_Y_PATH,train_y)
np.save(TEST_Y_PATH,test_y)

In [ ]:
# upload data
role = sagemaker.get_execution_role()
sess = sagemaker.session.Session()
train_X_uri = sess.upload_data(path=TRAIN_X_PATH, bucket=sess.default_bucket(), key_prefix=f'sagemaker/{project_name}')
train_y_uri = sess.upload_data(path=TRAIN_Y_PATH, bucket=sess.default_bucket(), key_prefix=f'sagemaker/{project_name}')
test_X_uri = sess.upload_data(path=TEST_X_PATH, bucket=sess.default_bucket(), key_prefix=f'sagemaker/{project_name}')
test_y_uri = sess.upload_data(path=TEST_Y_PATH, bucket=sess.default_bucket(), key_prefix=f'sagemaker/{project_name}')
print(train_X_uri)
print(train_y_uri)
print(test_X_uri)
print(test_y_uri)


In [ ]:
estimator = TensorFlow(
    entry_point='./src/train.py',
    role=role,
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    framework_version='2.4.1',
    py_version='py37',
    hyperparameters={
        'epochs':10,
    },
    tags = [{'Key': 'fassion-mnist-jobs', 'Value': '1'}]
)

In [ ]:
%%time

estimator.fit(
    {'train': train_X_uri[:-11]},
    experiment_config = {
        'TrialName' : demo_trial.trial_name,
        'TrialComponentDisplayName' : 'TrainingJob',
    }
)